# Yearly DDF cadence plots for one opsim

In [ ]:
import datetime
import schedview.collect
import schedview.plot
from rubin_sim.data import get_baseline
from lsst.resources import ResourcePath
from rubin_scheduler.utils import ddf_locations
from schedview import DayObs
from astropy.time import Time
import bokeh.io
import numpy as np
from IPython.display import display, Markdown

In [ ]:
bokeh.io.output_notebook(hide_banner=True)

In [ ]:
visits_fname = get_baseline()

In [ ]:
visits_rp = ResourcePath(visits_fname)
visits = schedview.collect.read_opsim(visits_rp, stackers=schedview.collect.visits.DDF_STACKERS)

In [ ]:
ddf_field_names = tuple(ddf_locations().keys())
# Different versions of the schedule include a DD: prefix, or not.
# Catch them all.
ddf_field_names = ddf_field_names + tuple([f"DD:{n}" for n in ddf_field_names])

# Figure out which column has the target names.
target_column_name = "target_name" if "target_name" in visits.columns else "target"
ddf_visits = visits.loc[visits[target_column_name].isin(ddf_field_names)]

In [ ]:
nightly_ddf = schedview.compute.visits.accum_teff_by_night(ddf_visits)
for year in np.arange(2025, 2036):
    start_mjd = int(Time(f"{year}-06-21").mjd)
    end_mjd = start_mjd + 365
    cadence_plots = schedview.plot.create_cadence_plot(nightly_ddf, start_mjd, end_mjd)
    display(Markdown(f"## DDF Cadence for {year}-{year+1}"))
    bokeh.io.show(cadence_plots)